In [25]:
import pandas as pd
import numpy as np
import os

In [35]:
# Read in player roster for the given season
current_season = '2023'

df_players = pd.read_csv(f'./data/players_{current_season}.txt')
df_players = df_players[['PLAYER_ID', 'PLAYER_NAME']]

for season in ['2022', '2021', '2020']:
    df_temp = pd.read_csv(f'./stats/per_game_stat_{season}.txt')
    df_temp = df_temp[['PLAYER_ID', 'RANK_H2H_DNP_ADJUSTED', 'RANK_DNP_ADJUSTED', 'RANK_H2H_RAW', 'RANK_RAW', 'TOTAL_RAW', 'TOTAL_DNP_ADJUSTED', 'H2H_RAW_WIN_PCT', 'H2H_DNP_ADJUSTED_WIN_PCT']]
    df_players = df_players.merge(df_temp, how='left', on=['PLAYER_ID'], suffixes=[None, f'_{season}'])

df_players['RANK_H2H_DNP_ADJUSTED_2020'] = np.where(df_players['RANK_H2H_DNP_ADJUSTED_2020'].isnull(), 120, df_players['RANK_H2H_DNP_ADJUSTED_2020'])
df_players['RANK_DNP_ADJUSTED_2020'] = np.where(df_players['RANK_DNP_ADJUSTED_2020'].isnull(), 120, df_players['RANK_DNP_ADJUSTED_2020'])
df_players['RANK_H2H_RAW_2020'] = np.where(df_players['RANK_H2H_RAW_2020'].isnull(), 120, df_players['RANK_H2H_RAW_2020'])
df_players['RANK_RAW_2020'] = np.where(df_players['RANK_RAW_2020'].isnull(), 120, df_players['RANK_RAW_2020'])

df_players['RANK_H2H_DNP_ADJUSTED_2021'] = np.where(df_players['RANK_H2H_DNP_ADJUSTED_2021'].isnull(), df_players['RANK_H2H_DNP_ADJUSTED_2020'], df_players['RANK_H2H_DNP_ADJUSTED_2021'])
df_players['RANK_DNP_ADJUSTED_2021'] = np.where(df_players['RANK_DNP_ADJUSTED_2021'].isnull(), df_players['RANK_DNP_ADJUSTED_2020'], df_players['RANK_DNP_ADJUSTED_2021'])
df_players['RANK_H2H_RAW_2021'] = np.where(df_players['RANK_H2H_RAW_2021'].isnull(), df_players['RANK_H2H_RAW_2020'], df_players['RANK_H2H_RAW_2021'])
df_players['RANK_RAW_2021'] = np.where(df_players['RANK_RAW_2021'].isnull(), df_players['RANK_RAW_2020'], df_players['RANK_RAW_2021'])

df_players['RANK_H2H_DNP_ADJUSTED'] = np.where(df_players['RANK_H2H_DNP_ADJUSTED'].isnull(), df_players['RANK_H2H_DNP_ADJUSTED_2021'], df_players['RANK_H2H_DNP_ADJUSTED'])
df_players['RANK_DNP_ADJUSTED'] = np.where(df_players['RANK_DNP_ADJUSTED'].isnull(), df_players['RANK_DNP_ADJUSTED_2021'], df_players['RANK_DNP_ADJUSTED'])
df_players['RANK_H2H_RAW'] = np.where(df_players['RANK_H2H_RAW'].isnull(), df_players['RANK_H2H_RAW_2021'], df_players['RANK_H2H_RAW'])
df_players['RANK_RAW'] = np.where(df_players['RANK_RAW'].isnull(), df_players['RANK_RAW_2021'], df_players['RANK_RAW'])

df_players['RANK_PRED_SCORE'] = 0.5/4*(df_players['RANK_H2H_DNP_ADJUSTED'] + df_players['RANK_H2H_RAW'] + df_players['RANK_DNP_ADJUSTED'] + df_players['RANK_RAW']) + 0.3/4*(df_players['RANK_H2H_DNP_ADJUSTED'] + df_players['RANK_H2H_RAW'] + df_players['RANK_DNP_ADJUSTED'] + df_players['RANK_RAW']) + 0.2/4*(df_players['RANK_H2H_DNP_ADJUSTED'] + df_players['RANK_H2H_RAW'] + df_players['RANK_DNP_ADJUSTED'] + df_players['RANK_RAW']) 
df_players = df_players.sort_values(by='RANK_PRED_SCORE', ascending=True, ignore_index=True).reset_index()
df_players = df_players.rename(columns={'index': 'RANK_PRED'})
df_players['RANK_PRED'] += 1

df_players = df_players[['PLAYER_ID', 'PLAYER_NAME', 'RANK_PRED'] + [col for col in df_players.columns if col.startswith('RANK_H2H_DNP_ADJUSTED')] + [col for col in df_players.columns if col.startswith('RANK_DNP_ADJUSTED')] + [col for col in df_players.columns if col.startswith('RANK_H2H_RAW')] + [col for col in df_players.columns if col.startswith('RANK_RAW')] + [col for col in df_players.columns if col.startswith('H2H')] + [col for col in df_players.columns if col.startswith('TOTAL')] + ['RANK_PRED_SCORE']]

df_players.to_excel(f'./fantasy/fantasy_predicted_ranking_{current_season}.xlsx', index=False)


In [34]:
# Practice rankings from previous seasons
seasons = [str(s) for s in range(2006,2023)]

for current_season in seasons:
    df_players = pd.read_csv(f'./stats/per_game_stat_{current_season}.txt')
    df_players = df_players[['PLAYER_ID', 'PLAYER_NAME', 'RANK_H2H_DNP_ADJUSTED', 'RANK_DNP_ADJUSTED', 'RANK_H2H_RAW', 'RANK_RAW']]

    for season in reversed([str(s) for s in range(int(current_season)-3,int(current_season))]):
        df_temp = pd.read_csv(f'./stats/per_game_stat_{season}.txt')
        df_temp = df_temp[['PLAYER_ID', 'RANK_H2H_DNP_ADJUSTED', 'RANK_DNP_ADJUSTED', 'RANK_H2H_RAW', 'RANK_RAW', 'TOTAL_RAW', 'TOTAL_DNP_ADJUSTED', 'H2H_RAW_WIN_PCT', 'H2H_DNP_ADJUSTED_WIN_PCT']]
        df_players = df_players.merge(df_temp, how='left', on=['PLAYER_ID'], suffixes=[None, f'_{season}'])

    df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-3}'] = np.where(df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-3}'].isnull(), 120, df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-3}'])
    df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-3}'] = np.where(df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-3}'].isnull(), 120, df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-3}'])
    df_players[f'RANK_H2H_RAW_{int(current_season)-3}'] = np.where(df_players[f'RANK_H2H_RAW_{int(current_season)-3}'].isnull(), 120, df_players[f'RANK_H2H_RAW_{int(current_season)-3}'])
    df_players[f'RANK_RAW_{int(current_season)-3}'] = np.where(df_players[f'RANK_RAW_{int(current_season)-3}'].isnull(), 120, df_players[f'RANK_RAW_{int(current_season)-3}'])

    df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-2}'] = np.where(df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-2}'].isnull(), df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-3}'], df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-2}'])
    df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-2}'] = np.where(df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-2}'].isnull(), df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-3}'], df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-2}'])
    df_players[f'RANK_H2H_RAW_{int(current_season)-2}'] = np.where(df_players[f'RANK_H2H_RAW_{int(current_season)-2}'].isnull(), df_players[f'RANK_H2H_RAW_{int(current_season)-3}'], df_players[f'RANK_H2H_RAW_{int(current_season)-2}'])
    df_players[f'RANK_RAW_{int(current_season)-2}'] = np.where(df_players[f'RANK_RAW_{int(current_season)-2}'].isnull(), df_players[f'RANK_RAW_{int(current_season)-3}'], df_players[f'RANK_RAW_{int(current_season)-2}'])

    df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-1}'] = np.where(df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-1}'].isnull(), df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-2}'], df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-1}'])
    df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-1}'] = np.where(df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-1}'].isnull(), df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-2}'], df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-1}'])
    df_players[f'RANK_H2H_RAW_{int(current_season)-1}'] = np.where(df_players[f'RANK_H2H_RAW_{int(current_season)-1}'].isnull(), df_players[f'RANK_H2H_RAW_{int(current_season)-2}'], df_players[f'RANK_H2H_RAW_{int(current_season)-1}'])
    df_players[f'RANK_RAW_{int(current_season)-1}'] = np.where(df_players[f'RANK_RAW_{int(current_season)-1}'].isnull(), df_players[f'RANK_RAW_{int(current_season)-2}'], df_players[f'RANK_RAW_{int(current_season)-1}'])

    df_players['RANK_PRED_SCORE'] = 0.5/4*(df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-1}'] + df_players[f'RANK_H2H_RAW_{int(current_season)-1}'] + df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-1}'] + df_players[f'RANK_RAW_{int(current_season)-1}']) + 0.3/4*(df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-2}'] + df_players[f'RANK_H2H_RAW_{int(current_season)-2}'] + df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-2}'] + df_players[f'RANK_RAW_{int(current_season)-2}']) + 0.2/4*(df_players[f'RANK_H2H_DNP_ADJUSTED_{int(current_season)-3}'] + df_players[f'RANK_H2H_RAW_{int(current_season)-3}'] + df_players[f'RANK_DNP_ADJUSTED_{int(current_season)-3}'] + df_players[f'RANK_RAW_{int(current_season)-3}']) 
    df_players = df_players.sort_values(by='RANK_PRED_SCORE', ascending=True, ignore_index=True).reset_index()
    df_players = df_players.rename(columns={'index': 'RANK_PRED'})
    df_players['RANK_PRED'] += 1

    df_players = df_players[['PLAYER_ID', 'PLAYER_NAME', 'RANK_PRED'] + [col for col in df_players.columns if col.startswith('RANK_H2H_DNP_ADJUSTED')] + [col for col in df_players.columns if col.startswith('RANK_DNP_ADJUSTED')] + [col for col in df_players.columns if col.startswith('RANK_H2H_RAW')] + [col for col in df_players.columns if col.startswith('RANK_RAW')] + [col for col in df_players.columns if col.startswith('H2H')] + [col for col in df_players.columns if col.startswith('TOTAL')] + ['RANK_PRED_SCORE']]
    df_players.to_excel(f'./fantasy/fantasy_predicted_ranking_{current_season}.xlsx', index=False)